# Variational monte carlo simulation for Hydrogen Molecule

In [71]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
# %matplotlib inline

### Function definitions

In [80]:
# Compute wavefunction
def wavefunc(r,pars):
    s = pars[0]; a = pars[1]; beta = pars[2];
    r1 = r[:,0:3]
    r2 = r[:,3:6]
    r1L = np.linalg.norm(r1 + np.array([s/2, 0, 0]),axis=1) 
    r1R = np.linalg.norm(r1 - np.array([s/2, 0, 0]),axis=1)
    r2L = np.linalg.norm(r2 + np.array([s/2, 0, 0]),axis=1) 
    r2R = np.linalg.norm(r2 - np.array([s/2, 0, 0]),axis=1)
    r12 = np.linalg.norm(r1-r2,axis=1)
    psi = (np.exp(-r1L/a) + np.exp(-r1R/a)) * (np.exp(-r2L/a) + np.exp(-r2R/a)) * np.exp(r12/(2*(1+beta*r12)))
    return psi

# Update N walkers 1 step
def updateWalker(r,N,d,pars):
    dis = np.random.normal(0,d,(N,6)) #proposed displacement
    p = (wavefunc(r+dis,pars)/wavefunc(r,pars))**2
    rnd = np.random.rand(N)
    r += (rnd<p)[:,None]*dis

# Compute local energy
def EL(r,pars):
    s = pars[0]; a = pars[1]; beta = pars[2];
    r1 = r[:,0:3]
    r2 = r[:,3:6]
    r1L = np.linalg.norm(r1 + np.array([s/2, 0, 0]),axis=1) 
    r1R = np.linalg.norm(r1 - np.array([s/2, 0, 0]),axis=1)
    r2L = np.linalg.norm(r2 + np.array([s/2, 0, 0]),axis=1) 
    r2R = np.linalg.norm(r2 - np.array([s/2, 0, 0]),axis=1)
    r12 = np.linalg.norm(r1-r2,axis=1)
    psi1L = np.exp(-r1L/a)
    psi1R = np.exp(-r1R/a)
    psi2L = np.exp(-r2L/a)
    psi2R = np.exp(-r2R/a)
    psi1 = psi1L+psi1R
    psi2 = psi2L+psi2R
    
    q1L = np.sum((r1 + np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r1L*r12)  #r1L_hat  dot  r12_hat
    q1R = np.sum((r1 - np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r1R*r12)  #r1R_hat  dot  r12_hat
    q2L = np.sum((r2 + np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r2L*r12)  #r2L_hat  dot  r12_hat
    q2R = np.sum((r2 - np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r2R*r12)  #r2R_hat  dot  r12_hat
    
    Eloc = -1/(a*a) + (psi1L/r1L + psi1R/r1R)/(a*psi1) + (psi2L/r2L + psi2R/r2R)/(a*psi2) - (1/r1L + 1/r1R + 1/r2L + 1/r2R) \
        + 1/r12 + ((psi1L*q1L+psi1R*q1R)/psi1 - (psi2L*q2L+psi2R*q2R)/psi2)/(2*a*(1+beta*r12)**2) \
        - ((4*beta+1)*r12+4)/(4*r12*(1+beta*r12)**4) + 1/s
    
    return Eloc

#Update variational parameters
def updateVarParams(R,EL,pars,gamma):
    beta = pars[2]
    R12 = np.linalg.norm(R[:,0:3]-R[:,3:6],axis=1)
    dlnpsida = -R12**2 / (2*(1+beta*R12)**2)
    dEda = 2 * (np.mean(EL*dlnpsida)-np.mean(EL)*np.mean(dlnpsida))
    beta -= gamma*dEda
    pars[2] = beta
    return pars, dEda

#One VMC simulation
def VMC(pars,N,T,T0,gamma):
    s = pars[0]; a = pars[1];
    r = np.random.uniform(-s-2*a,s+2*a,(N,6)) #init walkers
    R = np.empty((T*N,6))

    for t in range(0,T0):
        updateWalker(r,N,d,pars)

    for t in range(0,T):
        updateWalker(r,N,d,pars)
        R[t*N:(t+1)*N,:] = r

    Eloc = EL(R,pars)
    E = np.mean(Eloc)
    VarE = np.var(Eloc)  
        
    return E, VarE, Eloc, R

    

### Optimization routine over beta, for various s

In [192]:
N = 400 #Number of walkers
T = 26000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

d = 1/2 #mean step length
gamma = 10 #step size in gradient descent method

#Variational parameters
# alpha = 2 is hard-coded into all functions
srnge = np.arange(1.0,1.85,0.05)

#Parameter optimization variables
epsilon = 1e-4 # tolerance on derivative 
itmax = 10 #max number of optimization iterations

repeat = 5
res = [];
for s in srnge:
    beta = 0.6 #initial guess for beta
    impl_cond = lambda a : a*(1+np.exp(-s/a)) - 1
    a = fsolve(impl_cond,0.9)[0] #find a numerically using the above constraint
    pars = [s,a,beta]
    
    # Optimize for beta
    dEda = 2; it = 0;
    while((abs(dEda) >= epsilon) & (it<itmax)):
        E, VarE, Eloc,R = VMC(pars,N,T,T0,gamma)
        pars, dEda = updateVarParams(R,Eloc,pars,gamma)
        it=it+1
    
    # Repeat couple of times at optimal beta for error est.
    E = np.empty(repeat);    
    for i in range(repeat):
        E[i] = VMC(pars,N,T,T0,gamma)[0]
        
    res.append([np.mean(E), np.std(E), pars[0], pars[1], pars[2]])
        
res = np.array(res)
        

In [212]:
# Produce plot of s vs E at optimal beta
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.errorbar(res[:,2],res[:,0],yerr=2*res[:,1],color='k')
ax1.set_xlabel(r'$s$')
ax1.set_ylabel(r'$E$')
ax1.set_xlim(1,1.8)

ax2 = ax1.twinx()
ax2.plot(res[:,2],res[:,4],'r:x')
ax2.set_ylabel(r'$\beta$', color='r')
for tl in ax2.get_yticklabels():
    tl.set_color('r')
ax2.set_xlim(1,1.8)
plt.show()

### Routine that loops over s and beta (without optimization of parameters)


In [155]:
N = 200 #Number of walkers
T = 13000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

d = 1/2 #mean step length
gamma = 10 #step size in gradient descent method

#Variational parameters
# alpha = 2 is hard-coded into all functions
srnge = np.arange(1.0,1.85,0.05)
betarnge = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0, 1.2, 1.4]

repeat = 1
res = [];
for s in srnge:
    E = np.empty(repeat);
    for beta in betarnge:
        impl_cond = lambda a : a*(1+np.exp(-s/a)) - 1
        a = fsolve(impl_cond,0.9)[0] #find a from a numerically using the above constraint
        pars = [s,a,beta]
        
        for i in range(repeat):
            E[i] = VMC(pars,N,T,T0,gamma)[0]
        
        res.append([np.mean(E), np.std(E), s, a, beta])
        
res = np.array(res)
        

In [133]:
# Produce plot 2D for beta vs E at fixed s
fig = plt.figure()
plt.errorbar(res[:,4],res[:,0],yerr=2*res[:,1])
plt.xlabel(r'$\beta$')
plt.ylabel(r'$E$')
plt.show()

In [188]:
# Produce plot 3D for various beta and s
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(res[:,2], res[:,4], res[:,0], cmap=cm.jet, linewidth=0.2)
ax.set_xlim(1,1.8)
ax.set_ylim(0,1.4)
plt.xlabel(r'$s$')
plt.ylabel(r'$\beta$')
plt.show()
